# Using Producer Mini-App

In [ ]:
%%capture
# System Libraries
import sys, os
sys.path.append("..")
import pandas as pd
import numpy as np
import ast
import pykafka
import mass.kafka
 

## logging
import logging
logging.basicConfig(level=logging.DEBUG)
logging.getLogger().setLevel(logging.ERROR)
logging.getLogger("py4j").setLevel(logging.ERROR)
logging.getLogger("tornado.application").setLevel(logging.CRITICAL)
logging.getLogger("distributed.utils").setLevel(logging.CRITICAL)

import boto3
boto3.setup_default_session(profile_name='dev')  

# Pilot-Streaming
import pilot.streaming
import uuid 
import time

# 1. Benchmark Loop

In [ ]:
for num_repeats in range(1):
    for num_producer_nodes in [1]:
        
        dask_pilot_description = {
                 "resource":"slurm+ssh://login1.wrangler.tacc.utexas.edu",
                 "working_directory": os.path.join('/work/01131/tg804093/wrangler/', "work"),
                 "number_cores": 48*num_producer_nodes,
                 "project": "TG-MCB090174",
                 "queue": "normal",
                 "walltime": 300,
                 "type":"dask"
            }
        dask_pilot = pilot.streaming.PilotComputeService.create_pilot(dask_pilot_description)
        dask_pilot.wait()
        dask_details=dask_pilot.get_details()
        #dask_details={'master_url': 'tcp://c251-101:8786'}
        
        for num_broker_nodes in [1]: #,2,4
            for num_partitions_per_node in [16,32,64,128]:
                #for application in ["kmeans-5000", "kmeansstatic-5000", "kmeansstatic-10000", "kmeansstatic-20000", "light"]:
                for application_scenario in [ "light", "kmeans-5000"]:    
                    pilot_compute_description = {
                                                    "resource":"kinesis://awscloud.com",
                                                    "number_cores": num_partitions_per_node,
                                                    "type":"kinesis"
                                                }
                    kinesis_pilot = pilot.streaming.PilotComputeService.create_pilot(pilot_compute_description)
                    kinesis_pilot.wait()
                    kinesis_details=kinesis_pilot.get_details()
                    
                    time.sleep(5)
                    for number_parallel_tasks in [1]:
                        number_parallel_tasks = num_producer_nodes*number_parallel_tasks
                        number_points_per_message = 5000
                        if application_scenario.startswith("kmeans"):
                            number_points_per_message = int(application_scenario.split("-")[1])
                            application = application_scenario.split("-")[0]
                        else:
                            application = application_scenario
                        
                        print("Run Application: %s, Number Messages: %d"%(application, number_points_per_message))
                        run_id = str(uuid.uuid1())
                        miniapp=mass.kafka.MiniApp(
                                                     dask_scheduler=dask_details['master_url'],
                                                     resource_url=kinesis_details["master_url"],
                                                     broker_service="kinesis",
                                                     number_parallel_tasks=number_parallel_tasks,
                                                     number_clusters=10, # kmeans
                                                     number_points_per_cluster=10000, # kmeans
                                                     number_points_per_message=number_points_per_message, # kmeans
                                                     number_dim=3, # kmeans
                                                     number_messages=192, # light
                                                     number_produces=1,
                                                     number_partitions=num_broker_nodes*num_partitions_per_node,
                                                     topic_name="test-"+run_id,
                                                     application_type = application
                                                    )
                        miniapp.run()
                    try:
                        kinesis_pilot.cancel()
                        
                    except:
                        pass
        try:
            dask_pilot.cancel()
            time.sleep(60)
        except: 
            pass

In [ ]:
dask_pilot.get_details()

In [ ]:
import distributed
c=distributed.Client("tcp://c251-101:8786")

In [ ]:
c.scheduler_info()

In [ ]:
def inc(x):
    import socket
    return socket.gethostname()

In [ ]:
from dask.delayed import delayed
t = delayed(inc)(20)

In [ ]:
t.compute()

# 2. Mini App Test

Check Kafka Broker

In [ ]:
client = pykafka.KafkaClient(zookeeper_hosts=kafka_details["master_url"])
topic = client.topics['test']
producer = topic.get_sync_producer()
consumer = topic.get_simple_consumer()

In [ ]:
count = 0
number_total_points = 0
number_dimensions = 0
for i in range(100):
    message = consumer.consume(block=False)
    if message is not None:
        data_np = np.array(ast.literal_eval(message.value))
        num_points = data_np.shape[0]
        number_dimensions = data_np.shape[1]
        count =  count + 1
        number_total_points = number_total_points + num_points
    #print "Consumed message: %d, Number Points: %d, Number Dimensions: %d"%\
    #        (count, num_points, number_dimensions)   
        
print("Total Messages: %d, Total Points: %d, Number Dimensions: %d"%(count, number_total_points, number_dimensions))